# Training Bilderkennung Stoppschild

In diesem Notebook werden wir unser erstes Training mit den gelabelten Bildern der Stoppschild-Erkennung machen.

Das Notebook ist sehr ähnlich aufgebaut wie das von letzter Woche ([02c_Bilderkennung_mit_FastAI](../02_Homework/02c_Bilderkennung_mit_FastAI.ipynb)). Schaue, dass du das Notebook verstanden hast, da hier nicht mehr alles im Detail erklärt wird.

In [ ]:
from fastai.data.all import *
from fastai.vision.all import *

In [ ]:
# Sicherstellen, dass die GPU benützt wird, sonst geht das Training viel zu lange
setup_cuda()

# Als Ausgabe sollte folgendes stehen: Device selected: cuda:0
device = default_device()
print(f"Device selected: {device}")

## Daten sammeln

Wir haben die Daten schon im vorhergehenden Notebook gesammelt und gelabelt. Wenn nicht, hole das nach.

Danach kopiere die Daten auf dein Google Drive und verbinde es mit Colab (so wie du es letzte Woche gemacht hast).

In [ ]:
# Mit deinem Google Drive verbinden
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Pfad mit dem Ordner, wo die Bilder liegen. Ändere den Pfad, falls die Dateien bei dir an einem anderen Ort liegen.
image_directory_path = Path('/content/drive/MyDrive/AI-Challenge/daten/stoppschild-erkennung/train')

# Sollte der Pfad nicht existieren, wird hier eine Fehlermeldung ausgegeben
if not image_directory_path.exists():
    raise Exception(f"Fehler🛑 Der Pfad {image_directory_path} existiert nicht")
else:
    print(f"Alles gut👍 der Pfad {image_directory_path} wurde gefunden")

In [ ]:
# Wir sehen, ob alle Bilder gefunden werden, die richtige Grösse haben und schauen uns eins an

# alle Bildpfade laden
image_paths = get_image_files(image_directory_path)
print(f"Es wurden {len(image_paths)} Bilder gefunden")

# den ersten Pfad auswählen und ausgeben (wenn du einen anderen willst, ändere den Index)
some_image_path = image_paths[0]
print(some_image_path)

# Wir können das Label bestimmen je nach dem in welchem Ordner es liegt
print(f"Das Bild {some_image_path.name} hat das Label: {some_image_path.parent.name}")

# Bild laden und anzeigen
img = PILImage.create(some_image_path)
img.show()

# Prüfe die Grösse, diese sollte 160x120 betragen
if not img.size == (160,120):
    raise Exception(f"Fehler🛑 Bild hat die falsche Grösse (ist {img.size})")
else:
    print(f"Alles gut👍 Bild hat die korrekte Grösse von {img.size}")

## Daten aufbereiten

Auch hier müssen wir alle Bilder in einen `ImageDataloaders` laden.

In [ ]:
# Das Label ist der Ordnername in dem das Bild liegt
def label_func(filePath):
  return filePath.parent.name

dls = ImageDataLoaders.from_path_func(
    image_directory_path,
    get_image_files(image_directory_path),
    valid_pct=0.2, # 20% der Daten für die Validierung verwenden
    bs=64, # Batch-Size sollte immer eine 2-er Potenz sein (8, 16, 32, 64, 128,...)
    shuffle=True, # Mischen macht Sinn, damit die Batches Stoppschilder und keine Stoppschilder enthalten
    label_func=label_func,
    device=default_device() # benütze die GPU, falls vorhanden
    )

In [ ]:
# Wir können uns ein Batch anzeigen lassen (es werden nur die ersten 9 Bilder angezeigt)
dls.show_batch()

In [ ]:
# Wir sehen uns mal den Shape an
batch_data, batch_labels = dls.one_batch()
batch_data.shape

# Stelle dir auch hier die Frage, was bedeuten diese Zahlen?

In [ ]:
# Wir können uns das Vocabulary ansehen. Also welche Ausgabe welches Label bedeuten
dls.vocab

# Somit also 0=noStopSigns / 1=stopSigns

## Model definieren & trainieren

### Model definieren

Anders als letzte Woche, definieren wir nun unser Modell selbst. Dazu benützen wir die Pytorch-Library (`torch`), auf welche FastAI aufbaut. Mit `Sequential` geben wir an, dass die enthaltenen Layer alle nacheinander kommen sollen. Was die einzelnen Layer genau machen, ist im Theorie-Notebook dieser Woche beschrieben.


In [ ]:
# Wir definieren unsere eigene Architektur unseres neuronalen Netzwerks
model = torch.nn.Sequential(
            torch.nn.Conv2d(3,16,3),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(),
            torch.nn.Dropout2d(p=0.2),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),

            torch.nn.Conv2d(16,32,3),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(),
            torch.nn.Dropout2d(p=0.2),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),

            torch.nn.Conv2d(32,64,3),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(),
            torch.nn.Dropout2d(p=0.2),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),

            torch.nn.Conv2d(64,128,3),
            torch.nn.BatchNorm2d(128),
            torch.nn.ReLU(),
            torch.nn.Dropout2d(p=0.2),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),

            torch.nn.Conv2d(128,256,3),
            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU(),
            torch.nn.Dropout2d(p=0.2),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),

            torch.nn.Flatten(1,-1),
            torch.nn.Linear(768,256),
            torch.nn.BatchNorm1d(256),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),

            torch.nn.Linear(256,64),
            torch.nn.BatchNorm1d(64),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),

            torch.nn.Linear(64,32),
            torch.nn.BatchNorm1d(32),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),

            torch.nn.Linear(32,1),
            torch.nn.Sigmoid(),
        )

# Modell auf der GPU laufen lassen (falls vorhanden)
model.to(device);

### Learner erstellen

Wir definieren den `Learner` direkt und geben ihm alle benötigten Sachen mit, darunter unser Modell, welches wir erstellt haben

Als Metric verwenden wir hier anstatt der Genauigkeit (Accuracy) den `F1Score()`. Der F1-Score eignet sich in unserem Fall gut, da wir ein grosses Ungleichgewicht in den Daten haben. Das bedeutet, wir haben viel mehr Fotos ohne Stoppschilder als mit. Das Problem mit Accuracy und Ungleichgewicht ist, wenn das Modell stur behaupten würde, dass jedes Bild kein Stoppschild ist, so hätte es in den allermeisten Fällen recht und die Accuracy wäre hoch, obwohl es kein einziges Stoppschild erkannt hätte.

Die F1-Score hingegen kombiniert die Präzision (Precision) mit der Sensitivität (Recall) und ist so für unseren Fall viel aussagekräftiger. Falls du dich tiefer damit befassen willst, findest du hier eine ausführlich aber gut verständliche Erklärung:
- [www.python-kurs.eu/metriken.php](https://www.python-kurs.eu/metriken.php)

In [ ]:
# Wir definieren den F1-Score. Da unser Modell die Probability ausgibt, müssen wir diese noch runden:
f1score_raw = F1Score()
def f1score(true, pred, *args, **kwargs):
  true = true.round().to(torch.int)
  pred = pred.round().to(torch.int)
  return f1score_raw(true, pred)


# Wir definieren den Learner mit unserem Modell
learn = Learner(
    dls=dls, # Unsere Daten
    model=model, # Unser Modell
    loss_func=BCELossFlat(), # Als Loss-Funktion eignet sich BCELossFlat
    metrics=f1score, # Unsere F1-Score
)

In [ ]:
# Modell Architektur anschauen
learn.summary()

### Beste Lernrate finden

Wer letzte Woche das Notebook gut durchgearbeitet hat, der weiss noch, dass FastAI eine Funktion zur Verfügung stellt, mit der die ideale Lernrate ermittelt werden kann. Diese führt Probetrainings mit verschiedenen Lernraten durch. Die Kurve beginnt zuerst zu sinken und erreicht den idealen Wert etwa bevor sie wieder stark zu steigen beginnt.

In [ ]:
suggested_lr = learn.lr_find().valley

print(f"Die empfohlene Lernrate ist {suggested_lr}")

### Training

Für Training verwenden wir anstatt der `.fit()` Funktion die `.fit_one_cycle()`. Diese ist sehr ähnlich, hilft aber nochmals, das Training zu beschleunigen (respektive die Anzahl der benötigten Epochen zu verringern). Das funktioniert, da sie gemäss der 1Cycle-Policy die Lernrate verändert. Die Praxis hat gezeigt, dass dies besser funktioniert als mit einer konstanten Lernrate wie beim normalen `.fit()`.

In [ ]:
%%time

# Anzahl Epochen, die wir lernen wollen. Passe den Wert an wenn du willst
epochs = 15

# Beginne mit dem Lernen
learn.fit_one_cycle(
    n_epoch=epochs,
    lr_max=suggested_lr, # die vorher ermittelte Lernrate
    )

In [ ]:
# Nach dem Training können wir die Fehlerkurve anschauen und ermitteln, ob es zu einem Over/Underfitting gekommen ist.
learn.recorder.plot_loss()

## Validierung

Wir können aus der Lernkurve schon einiges ablesen, nun wollen wir uns noch einige andere Dinge ansehen.

In [ ]:
# (Diese Zeile ist nötig, damit show_results korrekt funktioniert. Ihr müsst die Details nicht verstehen)
import types
def show_results(self, ds_idx=1, dl=None, max_n=9, shuffle=True, **kwargs):
    if dl is None: dl = self.dls[ds_idx].new(shuffle=shuffle)
    b = dl.one_batch()
    _,_,preds = self.get_preds(dl=[b], with_decoded=True, act=lambda p: p.round().to(torch.int).squeeze()) # fix with act=
    dl.show_results(b, preds, max_n=max_n, **kwargs)

learn.show_results =  types.MethodType(show_results, learn) # batch

In [ ]:
# Ein paar (zufällige) Vorhersagen aus dem Validationset machen. Du kannst diese Zeile also mehrmals ausführen und bekommst immer andere Beispiele.
learn.show_results()

In [ ]:
# Ein einzelnes Bild vorhersagen

img = PILImage.create(some_image_path)
img.show()

label, probability, _ = learn.predict(img)
print(f"Das ist ein {label} mit einer Wahrscheinlichkeit von {probability.item():.0%}")


In [ ]:
# Und wir können auch den Interpreter benützten
interp = ClassificationInterpretation.from_learner(learn, act=lambda p: p.round().to(torch.int).squeeze())
interp.plot_top_losses(9, figsize=(15,10))

In [ ]:
# Wir können uns auch die Confusion Matrix anzeigen lassen.
# Wie diese genau funktioniert und was du dort herauslesen kannst findest du hier: https://www.python-kurs.eu/metriken.php

interp.plot_confusion_matrix(figsize=(5,5))

In [ ]:
# Wir können uns anstatt der absoluten Nummer auch die Genauigkeit in Prozent anzeigen lassen:
interp.plot_confusion_matrix(normalize=True, figsize=(5,5))

## Model benützen

Damit das Model auf dem Auto läuft, können wir es nicht wie schon gelernt mit `learn.save` abspeichern. (Du kannst `learn.save` trotzdem für das Training&Validierung nützen).

Stattdessen speichern wir es im **ONNX-Format** (Open Neural Network Exchange). Dieser Format macht es einfach neuronale Netze auszutauschen und auf einem anderen Gerät laufen zu lassen:

### ONNX Speichern

In [ ]:
# Wir müssen zuerst ONNX installieren
!pip install onnx --quiet

In [ ]:
# Check that no other transformers are present
dls.after_batch

In [ ]:
from PIL import Image
import torch
from torchvision import transforms
import onnx

## Wir definieren, wo wir das ONNX speichern wollen
onnx_export_filename = Path('/content/drive/MyDrive/AI-Challenge/models/stopsign_model.onnx')

## ONNX überschreibt die Datei falls sie schon existiert. Daher prüfen wir hier ob das der Fall ist um nicht versehentlich etwas zu überschreiben
if onnx_export_filename.exists():
  raise Exception(f"onnx Modell {onnx_export_filename} existiert schon - lösche es zuerst oder benenne es um, wenn du es behalten willst")

# Wir holen uns das Model aus dem Learner
model = learn.model

# Und versetzten es in den "Eval" Modus, damit ist es bereit für Vorhersagen
model.eval()

# Wir brauchen ein Beispiel-Bild um die Input-Shape für ONNX festzulegen.
with Image.open(some_image_path) as img:
    convert_tensor = transforms.ToTensor()
    input_tensor_example = convert_tensor(img).unsqueeze(0)
    # Hier können wir uns anschauen, wie die Shape aussieht. So weiss ONNX was es erwarten kann.
    print(img.shape, input_tensor_example.shape)


# Prüfen, ob wir mit GPU lernen oder ohne, da der Input davon abhängig ist
if default_device().type == 'cuda':
  input_tensor_example = input_tensor_example.cuda()

# Model speichern
torch.onnx.export(
    model,
    input_tensor_example,
    onnx_export_filename,
    input_names=["image"],
    output_names=["data"]
)

print(f"Model wurde als ONNX exportiert nach: {onnx_export_filename}")

In [ ]:
# Wir können mit diesen Zeilen prüfen, ob es korrekt exportiert wurde und auch wieder importiert werden kann
onnx_model = onnx.load(onnx_export_filename)
onnx.checker.check_model(onnx_model)

### ONNX laden und benützten

Wir schauen uns kurz an, wie man ein ONNX Modell laufen lassen kann. Das geschieht auch auf dem Auto.

In [ ]:
# Wir müssen die ONNX-Runtime installieren
! pip install onnxruntime~=1.15.1 --quiet

In [ ]:
from PIL import Image
from pathlib import Path
import torch
from torchvision import transforms
import onnxruntime as ort

# Wir laden unser gespeichertes ONNX
onnx_session = ort.InferenceSession(onnx_export_filename)

# Und packen den Ablauf für Vorhersagen in eine Funktion
def predict(image_path):
    # Wir müssen das Bild zuerst in das richtige Format bringen (das macht sonst FastAI für uns)
    with Image.open(image_path) as img:
        convert_tensor = transforms.ToTensor()
        tensor_img = convert_tensor(img).unsqueeze(0).numpy()

    # Hier starten wir die Vorhersage mit dem Bild als Input
    pred = onnx_session.run(None, {'image': tensor_img})

    return pred[0][0][0]

In [ ]:
# Wir können nun eine Vorhersage mit einem Bild machen
pred_percent = predict(some_image_path)

# Und das Resultat anzeigen, je nachdem welche Prozentzahl höher ist
print(f"Das Bild ist mit einer Wahrscheinlichkeit von {pred_percent:.0%}% ein Stoppschild")


# Zur Kontrolle, lassen wir uns das Bild noch anzeigen
PILImage.create(some_image_path)